# 과제 2: 실시간 추론용으로 모델 배포

## 과제 2.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies
import boto3
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

처리된 고객 데이터 집합을 검토합니다.

In [ ]:
#explore-dataset
column_list = ['income','age','workclass','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.dtypes
lab_test_data.head()

훈련 및 튜닝 실습에서 사용했던 모델을 기본 Amazon Simple Storage Service(Amazon S3) 버킷에 저장합니다. **create_model**을 사용하여 모델을 설정하고 훈련된 모델을 참조하도록 **ModelDataUrl**을 구성합니다.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## 과제 2.2: 실습에서 제공되는 합성 및 재훈련된 모델에서 엔드포인트 생성

Amazon SageMaker SDK for Python을 사용하여 엔드포인트를 생성할 때는 다음의 3단계를 수행합니다.
1. SageMaker 모델을 생성합니다.
2. HTTPS 엔드포인트용 엔드포인트 구성을 생성합니다.
3. HTTPS 엔드포인트를 생성합니다.

엔드포인트를 생성하는 방법에 관한 자세한 내용은 [엔드포인트 생성 및 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)를 참조하세요.

모델은 이미 생성했으므로 엔드포인트 구성과 엔드포인트를 생성할 수 있습니다. 

먼저 사용하려는 엔드포인트 구성 이름과 인스턴스 유형을 설정합니다. 그런 다음 CreateEndpointConfig API를 호출합니다.

엔드포인트 구성을 생성하려면 다음 옵션을 설정해야 합니다.
- **VariantName**: 프로덕션 변형(프로덕션 환경의 모델 하나 이상)의 이름입니다.
- **ModelName**: 호스팅할 모델의 이름입니다. 모델을 생성할 때 지정한 이름을 사용하면 됩니다.
- **InstanceType**: 컴퓨팅 인스턴스 유형입니다.
- **InitialInstanceCount**: 처음 시작할 인스턴스의 수입니다.

엔드포인트의 입력과 SageMaker 실시간 엔드포인트의 추론 출력을 Amazon S3에 로깅하려는 경우 데이터 캡처 기능을 사용하도록 설정할 수 있습니다. 데이터 캡처는 일반적으로 훈련, 디버깅 및 모니터링에 사용할 수 있는 정보를 기록하는 데 사용됩니다. 데이터 캡처가 사용하도록 설정되어 있으면 Amazon SageMaker Studio에서 엔드포인트를 탐색할 때 엔드포인트 관련 추가 세부 정보가 표시됩니다. 이 실습 뒷부분에서 데이터 캡처 기능을 구성하여 데이터 캡처를 사용하도록 설정하는 방법을 확인합니다.

데이터 캡처 기능을 추가하는 방법에 관한 자세한 내용은 [데이터 캡처](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html)를 참조하세요.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date  
# so it you can search endpoints based on creation time.
endpoint_config_name = 'income-model-real-time-endpoint-{}'.format(create_date)                              
instance_type = 'ml.m5.xlarge'   
initial_sampling_percentage = 25 # Choose a value between 0 and 100
capture_modes = [ "Input",  "Output" ] # Specify input, output, or both

endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    DataCaptureConfig= {
        'EnableCapture': True, # Whether data should be captured or not.
        'InitialSamplingPercentage' : initial_sampling_percentage,
        'DestinationS3Uri': f's3://{bucket}/data-capture',
        'CaptureOptions': [{"CaptureMode" : capture_mode} for capture_mode in capture_modes]
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

다음으로는 엔드포인트를 생성합니다. 실시간 엔드포인트를 생성하면 SageMaker가 기계 학습(ML) 컴퓨팅 인스턴스를 시작하고 구성에 지정된 대로 모델을 하나 이상 배포합니다. 이 실습에서는 추론용 모델 하나만 배포합니다. SageMaker에서는 다중 모델 엔드포인트를 생성할 수 있습니다. 다중 모델 엔드포인트에 관한 자세한 내용은 [다중 모델 엔드포인트 호출](https://docs.aws.amazon.com/sagemaker/latest/dg/invoke-multi-model-endpoint.html)을 참조하세요.

엔드포인트가 작동 중이면 Helper 함수는 엔드포인트 Amazon 리소스 이름(ARN)을 인쇄합니다. 엔드포인트 생성을 실행하려면 약 5-7분이 소요됩니다.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


SageMaker Studio의 **Endpoints** 탭에서 엔드포인트 세부 정보를 검토할 수 있습니다.

다음 단계에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **real_time_inference_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **real_time_inference_ko_kr.ipynb** 탭을 선택한 다음, 도구 모음에서 **File**과 **New View for Notebook**을 선택합니다. 그러면 엔드포인트를 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 엔드포인트 탐색을 완료한 후 **real_time_inference_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. **SageMaker Home** 아이콘을 선택합니다.
2. **Deployments**를 선택해서 확장합니다.
3. **Endpoints**를 선택합니다.

SageMaker Studio에서 **Endpoints** 탭이 표시됩니다.

4. **Name** 열에 **income-model-real-time-** 이 있는 엔드포인트를 선택합니다.

엔드포인트가 표시되지 않으면 목록에 엔드포인트가 나타날 때까지 새로 고침 아이콘을 선택합니다.

SageMaker Studio에서 **ENDPOINT DETAILS** 탭이 표시됩니다.

5. **AWS settings** 탭을 선택합니다.

생성이 완료되기 전에 엔드포인트를 열었다면 **Endpoint status**가 *Creating*에서 *InService*로 바뀔 때까지 새로 고침 아이콘을 선택합니다.

**Endpoint type**이 **Real-time**으로 표시됩니다. **Data capture settings** 및 **Endpoint configuration settings** 섹션에는 이전 과제를 수행할 때 노트북에서 선택한 구성이 표시됩니다.

## 과제 2.3: 실시간 고객 레코드를 사용하는 실시간 추론을 위해 엔드포인트 호출

SageMaker 호스팅 서비스를 사용하여 모델을 배포한 후에는 엔드포인트로 테스트 데이터를 전송하여 해당 엔드포인트에서 모델을 테스트할 수 있습니다.

이 실습에서 사용하는 고객 레코드 중에는 소득이 미화 5만 달러 이상인 레코드(**income** 값: **1**)도 있고 5만 달러 미만인 레코드(**income** 값: **0**)도 있습니다. 이러한 레코드가 포함된 엔드포인트를 호출하여 반환되는 점수를 확인합니다.

엔드포인트에서 실시간 예측을 확인하려는 경우 응답에서 반환되는 본문 텍스트를 읽습니다. 이 텍스트에는 예측 점수 목록이 포함되어 있습니다. 각 레코드의 점수는 **0**에서 **1** 사이입니다. 값이 **1**에 가까울수록 해당 고객의 소득이 미화 5만 달러 이상일 가능성이 높은 것입니다. 예를 들어 예측 점수가 **0.42**인 고객은 예측 점수가 **0.14**인 고객보다 소득이 미화 5만 달러 이상일 가능성이 높습니다.

In [ ]:
#invoke-endpoint-real-time-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('56,3,6,6,0,3,1,0,0,1,0,13\n' +
                '29,2,2,2,0,1,0,0,0,0,0,70\n' +
                '79,0,1,1,0,3,5,0,0,0,0,20\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,3,2,0,0,0,0,32\n' +
                '31,0,1,1,2,1,2,1,1,0,0,40\n' +
                '23,0,1,1,1,0,1,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## 과제 2.4: 엔드포인트 삭제

다음의 3단계를 수행하여 엔드포인트를 정리할 수 있습니다. 먼저 엔드포인트를 삭제합니다. 그런 다음, 엔드포인트 구성을 삭제합니다. 마지막으로, 배포한 모델이 더 이상 필요하지 않다면 모델을 삭제합니다.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 마무리

축하합니다! SageMaker에서 SageMaker Python SDK를 사용하여 실시간 엔드포인트를 생성했으며 해당 엔드포인트를 호출했습니다.

이 실습의 다음 과제에서는 서버리스 추론을 사용하는 추론용 모델을 배포하는 과정을 중점적으로 살펴봅니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아와 **과제 3: 서버리스 추론용으로 모델 배포**를 계속 진행합니다.